In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os, re
from scipy.optimize import curve_fit
from scipy import stats
import SPTCata as spt
from collections import defaultdict

# 2019-09-12 Three species diffusion model
In this (short) notebook I want to explore the possibility that a three-species diffusion model fits better the data.

In [ ]:
# load all the trajectories
spt_rootdir = '/home/rcortini/work/CRG/projects/catadata/data'
exp_names = [
    '0_Tannic_Acid_6h',
    '1_Olaparib_R5020',
    '2_DMSO_R5020_Control',
    '3_EtOH_Nohormone_Control'
]
bad_cells = [
 '3_EtOH_Nohormone_Control/Stack019_Cell3',
 '2_DMSO_R5020_Control/Stack015_Cell1',
 '3_EtOH_Nohormone_Control/Stack003_Cell1',
 '3_EtOH_Nohormone_Control/Stack024_Cell1',
 '2_DMSO_R5020_Control/Stack004_Cell3',
 '3_EtOH_Nohormone_Control/Stack021_Cell3',
 '3_EtOH_Nohormone_Control/Stack024_Cell2',
 '2_DMSO_R5020_Control/Stack007_Cell2',
 '2_DMSO_R5020_Control/Stack001_Cell2',
 '3_EtOH_Nohormone_Control/Stack018_Cell2',
 '1_Olaparib_R5020/Stack007_Cell3'
]
experiments = spt.load_experiments(spt_rootdir, exp_names, bad_cells=bad_cells, quality=50)

Once all trajectories are loaded, let's calculate the values of $r^2$ to then perform the fitting.

In [ ]:
r2 = defaultdict(list)
for experiment_name, cells in experiments.items() :
    for cell in cells :
        for trajectory in cell :
            r = np.linalg.norm(np.diff(trajectory, axis=0), axis=1)
            r2[experiment_name].extend(r**2)

Let's define now the three-species diffusion model.

In [ ]:
def diffusion_model_3s(X, *p) :
    """
    Two species diffusion model
    """
    # let's extract data and contants
    x = X[0]         # data to fit
    dt = X[1]        # Delta t values
    
    # extract the parameters
    D1 = p[0]
    D2 = p[1]
    D3 = p[2]
    f1 = p[3]
    f2 = p[4]
    
    # f3 is the fraction of the rest
    f3 = 1-f1-f2
    
    # calculate the result of the model
    y = 1-f1*np.exp(-x/(4*D1*dt))-\
          f2*np.exp(-x/(4*D2*dt))-\
          f3*np.exp(-x/(4*D3*dt))
    return y

In [ ]:
# define the minimum and maximum value of the square displacement that we want
# to study
m = 0
M = 1.1

# number of points to include in the final histogram
N = 100

# the values of x in the fit are independent of the experiment
bins = np.linspace(m, M, N)
x = np.zeros(N)
x[1:] = bins[1:]

# constants for fitting
dt = 0.015         # in seconds... = 15ms
X = (x, dt)

# calculate the histogram of square displacements
y = {}
for exp_name, r2vals in r2.items() :
    # calculate the histogram of r2 values
    h, bins = np.histogram(r2vals, bins=bins)
    y[exp_name] = np.zeros(N)
    y[exp_name][1:] = np.cumsum(h)/sum(h)
    
    # fit
    p0 = [5.0, 1.0, 0.1, 0.3, 0.3]
    coeffs, var_matrix = curve_fit(diffusion_model_3s, X, y[exp_name], p0=p0,
                                       bounds = (0,
                                                 [np.inf, np.inf, np.inf, 1.0, 1.0]))
    plt.plot(x,y[exp_name], 'o', color='b')
    plt.plot(x, diffusion_model_3s((x, dt), *coeffs), color='b')
    plt.title(exp_name.lstrip(spt_rootdir))
    plt.xlabel("Square displacement [microns 2]", fontsize=18)
    plt.ylabel("Cumulative distribution", fontsize=18)
    plt.text(0.6, 0.2, "D1 = %.3f $\mu m^2/s$\n"
                         "D2 = %.3f $\mu m^2/s$\n"
                         "D3 = %.3f $\mu m^2/s$\n"
                         "f1 = %.3f\n"
                         "f2 = %.3f\n"
                         "f3 = %.3f\n"%(coeffs[0],coeffs[1],coeffs[2],
                                        coeffs[3],coeffs[4],1-coeffs[3]-coeffs[4]), fontsize = 14)
    plt.show()

In [ ]:
coeffs